In [1]:
# import torch as th

import matplotlib.pyplot as plt
import time

import sys
import os
from __future__ import division
import os
import urllib, cStringIO
import pymongo as pm ## first establish ssh tunnel to server where database is running
import base64
import numpy as np
from numpy import *
import PIL
from PIL import Image
import base64
import matplotlib
%matplotlib inline
from matplotlib import pylab, mlab, pyplot
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')
from matplotlib.path import Path
import matplotlib.patches as patches
import cv2
import pandas as pd
from svgpathtools import parse_path
import svg_distance_helpers as rsh
# import torch
# from torch.autograd import Variable
# import SimpleITK as sitk
# from torch.autograd import Variable


#### File Hierarchy

In [2]:
# directory & file hierarchy
analysis_dir = os.getcwd()
data_dir = os.path.join(analysis_dir,"tracing_overlap")
ref_dir = os.path.join(analysis_dir, 'tracing_ref_400')
tracing = ['square','shape', 'this circle']

#### Set Up

In [3]:
# generate reference shapes
really_run_this = 1
if really_run_this:
    ref_path = []
    for t in tracing:
        print t
        fname = os.path.join(ref_dir, '{}.png'.format(t))
        img_ref = cv2.imread(fname)
        img_ref = rsh.color_to_num(img_ref, threshold=0, white_background=False)
        ofname = os.path.join(ref_dir, '{}_ref.png'.format(t))
        cv2.imwrite(ofname, 255-img_ref*255)
        ref_path.append(ofname)
    

square
shape
this circle


In [ ]:
# Overlap the drawing on reference images
iteration_name = ['CDM_photodraw_e1', 'cdm_run_v3', 'cdm_run_v4']# name labels used in kiddraw database
dir_name = ['photodraw_e1', 'museumstation_v3', 'museumstation_v4'] # name labels will be used for new data storage 
tracing = ['square', 'shape', 'this circle']

# create the folder to store overlapped images
overlap_dir = os.path.join(analysis_dir, 'tracing_overlap')
if not os.path.exists(overlap_dir):
    os.makedirs(overlap_dir)

for di, dname in enumerate(dir_name):
    # create an individual folder for each experiment+version pair
    iter_dir = os.path.join(overlap_dir, dname)
    if not os.path.exists(iter_dir):
        os.makedirs(iter_dir)
        
    current_data_dir = os.path.join(analysis_dir,"tracing_{}".format(iteration_name[di]))

    # get tracings from the data folder
    for ti, t in enumerate(tracing):
            
        # folders that store the original tracing files
        if dname == 'photodraw_e1' and (t == 'square' or t == 'shape'):
            t = 'this ' + t
        t_dir = os.path.join(current_data_dir, t)
        
        # create folders to store overlapped images
        overlap_t = os.path.join(iter_dir, t.replace('this ', ''))
        if not os.path.exists(overlap_t):
                os.makedirs(overlap_t)
        
        
        if os.path.exists(t_dir):
            for fname in os.listdir(t_dir):
                if fname.startswith('age'):
                    img_draw = os.path.join(t_dir, fname)
                    img_ref = ref_path[ti]

                    draw = cv2.imread(img_draw).astype(int16)
                    ref = cv2.imread(img_ref).astype(int16)
                    img = ref
                    img[np.sum(ref, axis=2)<255*3] = [150, 150, 150]
                    img[np.sum(draw, axis=2)<255*3] = [40, 40, 170]

                    split_fname = fname.split('_')
                    age = split_fname[0]
                    timestamp = split_fname[3][2:]
                    new_fname = "{}_{}_{}_{}.png".format(age, dname, timestamp, t.replace('this ', ''))
                    cv2.imwrite(os.path.join(iter_dir,t.replace('this ', ''), new_fname), img)

#### upload stims to s3

Execute python script at command line by running: `python upload_stims_to_s3.py --bucket_name kiddraw-tracing --path_to_imgs ./tracing_eval_stims`

#### build stimulus dictionary & upload metadata to mongo

In [4]:
# read in data from csv files
cdm_v3 = pd.read_csv('MuseumStation_Tracing_Descriptives_cdm_run_v3.csv')
cdm_v4 = pd.read_csv('MuseumStation_Tracing_Descriptives_cdm_run_v4.csv')
photodraw = pd.read_csv('Photodraw_Tracing_Descriptives_CDM_photodraw_e1.csv')
limit = 200
overall = photodraw
#overall = pd.concat([overall, cdm_v3, cdm_v4])

for t in tracing:
    for a in range(2,11):
        cdm_v3_tage = cdm_v3.loc[(cdm_v3['age'] == 'age{}'.format(a)) & (cdm_v3['category']== t)]
        v3_size = cdm_v3_tage.shape[0]                 
        if v3_size > limit:
            overall = pd.concat([overall, cdm_v3_tage[:limit]])
            
        else:
            overall = pd.concat([overall, cdm_v3_tage]) 
            
            cdm_v4_tage = cdm_v4.loc[(cdm_v4['age'] == 'age{}'.format(a)) & (cdm_v4['category']== t)]
            
            overall = pd.concat([overall, cdm_v4_tage[:limit-v3_size]])

print overall.shape
for t in tracing:
    for a in range(2,11):
        print t, a, overall.loc[(overall['age'] == 'age{}'.format(a)) & ( (overall['category'] == t) | (overall['category'] == 'this {}'.format(t)) )].shape

overall = overall.drop(['filename', "Unnamed: 0"], axis=1)
overall.reset_index(inplace = True)

(5251, 14)
square 2 (200, 14)
square 3 (200, 14)
square 4 (212, 14)
square 5 (224, 14)
square 6 (215, 14)
square 7 (209, 14)
square 8 (200, 14)
square 9 (169, 14)
square 10 (200, 14)
shape 2 (200, 14)
shape 3 (200, 14)
shape 4 (211, 14)
shape 5 (224, 14)
shape 6 (215, 14)
shape 7 (209, 14)
shape 8 (200, 14)
shape 9 (137, 14)
shape 10 (200, 14)
this circle 2 (200, 14)
this circle 3 (200, 14)
this circle 4 (213, 14)
this circle 5 (224, 14)
this circle 6 (215, 14)
this circle 7 (159, 14)
this circle 8 (155, 14)
this circle 9 (95, 14)
this circle 10 (165, 14)


In [5]:
overall['exp_name'] = 0
overall['iteration_name'] = 0
overall['timestamp'] = 0
overall['img_url'] = 0
overall['number_rating_levels'] = 5
overall['lower_bound'] = 'poor'
overall['upper_bound'] = 'excellent' 

for index, row in overall.iterrows():
    sid = row['session_id']    
    
    if sid.startswith('CDM'):
        overall.loc[index,'exp_name'] = 'photodraw'
        overall.loc[index,'iteration_name'] = 'e1'
    else:
        overall.loc[index,'exp_name'] = 'museumstation'
        overall.loc[index,'iteration_name'] = sid.split('_')[2][:2]
    
    timestamp = row['session_id'].split('_')[-1][2:]
    overall.loc[index, 'timestamp'] = timestamp
    category = row['category'].replace('this ','')
    overall.loc[index,'category'] = category
    
    fname = '{}_{}_{}_{}_{}.png'.format(row['age'], overall.loc[index,'exp_name'],overall.loc[index,'iteration_name'], timestamp, category)
    
    folder_name = '{}_{}'.format(overall.loc[index,'exp_name'], overall.loc[index,'iteration_name'])
    render_fname = os.path.join(data_dir, folder_name, category, fname)
    
    if not os.path.exists(render_fname):
        print index, render_fname
        overall = overall.drop(index)
        continue
    
    overall.loc[index,'img_url'] = 'https://s3.amazonaws.com/{}/{}'.format('kiddraw-tracing',fname)
    overall.loc[index,'games'] = '[]'
    overall.loc[index,'age'] = overall.loc[index,'age'].replace('age','')

overall['age'] = pd.to_numeric(overall['age'])


3 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/photodraw_e1/square/age4_photodraw_e1_1532113637303_square.png
27 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/photodraw_e1/square/age4_photodraw_e1_1532467421953_square.png
28 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/photodraw_e1/circle/age4_photodraw_e1_1532467421953_circle.png
60 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/photodraw_e1/shape/age4_photodraw_e1_1532547528521_shape.png
76 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/photodraw_e1/shape/age4_photodraw_e1_1532555203616_shape.png
81 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/photodraw_e1/square/age5_photodraw_e1_1532556623648_square.png
83 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/phot

326 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age2_museumstation_v3_1528495589660_square.png
327 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age2_museumstation_v3_1528496399060_square.png
329 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age2_museumstation_v3_1528568736238_square.png
330 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age2_museumstation_v3_1528570199303_square.png
331 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age2_museumstation_v3_1528583999112_square.png
332 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age2_museumstation_v3_1528664124624_square.png
333 /Users/zixianchai/Desktop/comp

484 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age3_museumstation_v3_1528225497824_square.png
493 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age3_museumstation_v3_1528227854755_square.png
496 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age3_museumstation_v3_1528310999266_square.png
499 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age3_museumstation_v3_1528400999230_square.png
504 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age3_museumstation_v3_1528483799182_square.png
510 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age3_museumstation_v3_1528493399220_square.png
512 /Users/zixianchai/Desktop/comp

643 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age4_museumstation_v3_1527976799200_square.png
649 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age4_museumstation_v3_1528154999081_square.png
653 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age4_museumstation_v3_1528316999098_square.png
674 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age4_museumstation_v3_1528581292853_square.png
679 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age4_museumstation_v3_1528662866503_square.png
686 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age4_museumstation_v3_1528845188306_square.png
689 /Users/zixianchai/Desktop/comp

976 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age5_museumstation_v3_1532460062811_square.png
981 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age6_museumstation_v3_1525890491721_square.png
994 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age6_museumstation_v3_1526063910231_square.png
1005 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age6_museumstation_v3_1526409936551_square.png
1017 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age6_museumstation_v3_1527787275455_square.png
1038 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/square/age6_museumstation_v3_1528405799163_square.png
1047 /Users/zixianchai/Desktop/

...

3737 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age2_museumstation_v3_1527972599108_circle.png
3739 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age2_museumstation_v3_1528059244623_circle.png
3746 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age2_museumstation_v3_1528238432992_circle.png
3748 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age2_museumstation_v3_1528321018911_circle.png
3752 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age2_museumstation_v3_1528323599125_circle.png
3756 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age2_museumstation_v3_1528480199337_circle.png
3757 /Users/zixianchai/Deskt

3942 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age3_museumstation_v3_1527971399240_circle.png
3945 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age3_museumstation_v3_1527982799096_circle.png
3948 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age3_museumstation_v3_1528061999122_circle.png
3949 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age3_museumstation_v3_1528063199159_circle.png
3954 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age3_museumstation_v3_1528139252267_circle.png
3955 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age3_museumstation_v3_1528140599196_circle.png
3956 /Users/zixianchai/Deskt

4114 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age4_museumstation_v3_1526765744803_circle.png
4118 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age4_museumstation_v3_1527717845451_circle.png
4119 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age4_museumstation_v3_1527719230278_circle.png
4122 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age4_museumstation_v3_1527882194414_circle.png
4128 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age4_museumstation_v3_1527969790348_circle.png
4129 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age4_museumstation_v3_1527970799228_circle.png
4133 /Users/zixianchai/Deskt

4418 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age5_museumstation_v3_1531773335311_circle.png
4427 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age5_museumstation_v3_1531953239905_circle.png
4441 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age5_museumstation_v3_1532548833351_circle.png
4449 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age5_museumstation_v3_1532639150758_circle.png
4477 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age5_museumstation_v3_1533578333379_circle.png
4480 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age5_museumstation_v3_1533588293512_circle.png
4485 /Users/zixianchai/Deskt

5135 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age10_museumstation_v3_1530652167310_circle.png
5154 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age10_museumstation_v3_1531850661804_circle.png
5169 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age10_museumstation_v3_1532126009312_circle.png
5177 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age10_museumstation_v3_1532634445345_circle.png
5185 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age10_museumstation_v3_1532807946580_circle.png
5190 /Users/zixianchai/Desktop/compositionality/shapedraw/analysis/tracing/tracing_overlap/museumstation_v3/circle/age10_museumstation_v3_1533151204661_circle.png
5200 /Users/zixianchai

In [6]:
try:
    overall = overall.drop('index', axis=1)
except:
    pass

bin_limit = 80
for t in ['square', 'shape', 'circle']:
    for a in range(2,11):
        drop_index = overall.index[(overall['age'] == a) & (overall['category'] == t)].tolist()[bin_limit:]
        overall = overall.drop(drop_index)
        print t, a, overall.loc[(overall['age'] == a) & ( (overall['category'] == t) | (overall['category'] == 'this {}'.format(t)) )].shape



square 2 (80, 20)
square 3 (80, 20)
square 4 (80, 20)
square 5 (80, 20)
square 6 (80, 20)
square 7 (80, 20)
square 8 (80, 20)
square 9 (80, 20)
square 10 (80, 20)
shape 2 (80, 20)
shape 3 (80, 20)
shape 4 (80, 20)
shape 5 (80, 20)
shape 6 (80, 20)
shape 7 (80, 20)
shape 8 (80, 20)
shape 9 (80, 20)
shape 10 (80, 20)
circle 2 (80, 20)
circle 3 (80, 20)
circle 4 (80, 20)
circle 5 (80, 20)
circle 6 (80, 20)
circle 7 (80, 20)
circle 8 (80, 20)
circle 9 (80, 20)
circle 10 (80, 20)


In [7]:
## apply weird literal eval hack to convert string into regular list
import ast
overall['games'] = '[]'
overall['games'] = overall['games'].apply(lambda x: ast.literal_eval(x))
print overall.shape

(2160, 20)


In [8]:
# insert shuffler_ind
data_range = np.arange(overall.shape[0])
np.random.shuffle(data_range)
overall['shuffler_ind'] = data_range

In [9]:
try:
    overall = overall.drop('index', axis=1)
except:
    pass

## save out data frame as csv for our own records
overall.to_csv('kiddraw_tracing_eval.csv',index=False)

In [10]:
## convert 'overall' pandas dataframe to json object and save out
import json
stimdict = overall.to_dict(orient='records') 
with open('{}.js'.format('kiddraw_tracing_eval'), 'w') as fout:
    json.dump(stimdict, fout)

### load JSON back in to inspect before uploading to mongo

In [11]:
### define datasetname
dataset_name = 'kiddraw_tracing_eval'

### next todo is to upload this JSON to initialize the new stimulus collection
print('next todo is to upload this JSON to initialize the new stimulus collection...')
import json
J = json.loads(open('{}.js'.format(dataset_name),mode='ru').read())

next todo is to upload this JSON to initialize the new stimulus collection...


In [12]:
##assert len(J)==len(all_files)
print 'dataset_name: {}'.format(dataset_name)
print len(J)

dataset_name: kiddraw_tracing_eval
2160


In [13]:
## look at one example record
J[0]

{u'age': 7,
 u'bounding_box': u'(96, 287, 115, 300)',
 u'category': u'square',
 u'draw_duration_new': 0.0,
 u'draw_duration_old': 0.0,
 u'exp_name': u'photodraw',
 u'games': [],
 u'img_url': u'https://s3.amazonaws.com/kiddraw-tracing/age7_photodraw_e1_1532111112405_square.png',
 u'iteration_name': u'e1',
 u'lower_bound': u'poor',
 u'mean_intensity': 0.01055,
 u'num_strokes': 1,
 u'number_rating_levels': 5,
 u'session_id': u'CDM_photodraw_e11532111112405',
 u'shuffler_ind': 575,
 u'submit_date': u'2018-07-20T19:04:03.404Z',
 u'submit_time': 1532113443404.0,
 u'timestamp': u'1532111112405',
 u'trial_duration': nan,
 u'trial_num': 0,
 u'upper_bound': u'excellent'}

In [22]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['stimuli']
coll = db[dataset_name]
dev_name = 'kiddraw_tracing_eval_dev'
dev_coll = db[dev_name]

In [15]:
reallyRun = 1
if reallyRun:
    coll.drop()
print 'Currently there are {} tracings in the production collection'.format(coll.count())

Currently there are 0 tracings in the production collection


In [21]:
from IPython.display import clear_output
## actually add data now to the database
reallyRun = 1
if reallyRun:
    for (i,j) in enumerate(J):
        if i%1==0:
            print ('%d of %d' % (i,len(J)))
            clear_output(wait=True)
        coll.insert(j)

2159 of 2160


In [26]:
reallyRun = 1
if reallyRun:
    dev_coll.drop()
print 'Currently there are {} tracings in the development collection'.format(dev_coll.count())

Currently there are 0 tracings in the development collection


In [27]:
# set the dev collection
reallyRun = 1
if reallyRun:
    for (i,j) in enumerate(J):
        if i%1==0:
            print ('%d of %d' % (i,len(J)))
            clear_output(wait=True)
        dev_coll.insert(j)

2159 of 2160


In [28]:
## check how many records have been retrieved
print 'Currently there are {} tracings in the production collection'.format(coll.count())
print 'Currently there are {} tracings in the development collection'.format(dev_coll.count())

a = coll.find({'num_strokes':{'$gte':0}}).sort('timestamp')
numGames = []
for rec in a:
    numGames.append(len(rec['games']))
b = np.array(numGames)
print np.mean(b>0)

Currently there are 2160 tracings in the production collection
Currently there are 2160 tracings in the development collection
0.0


In [29]:
print "Currentely, there are {} number of tracing stimuli in the database".format(coll.count())
coll.find_one()

Currentely, there are 2160 number of tracing stimuli in the database


{u'_id': ObjectId('5c06d7231943e66bf5cded02'),
 u'age': 7,
 u'bounding_box': u'(96, 287, 115, 300)',
 u'category': u'square',
 u'draw_duration_new': 0.0,
 u'draw_duration_old': 0.0,
 u'exp_name': u'photodraw',
 u'games': [],
 u'img_url': u'https://s3.amazonaws.com/kiddraw-tracing/age7_photodraw_e1_1532111112405_square.png',
 u'iteration_name': u'e1',
 u'lower_bound': u'poor',
 u'mean_intensity': 0.01055,
 u'num_strokes': 1,
 u'number_rating_levels': 5,
 u'session_id': u'CDM_photodraw_e11532111112405',
 u'shuffler_ind': 575,
 u'submit_date': u'2018-07-20T19:04:03.404Z',
 u'submit_time': 1532113443404.0,
 u'timestamp': u'1532111112405',
 u'trial_duration': nan,
 u'trial_num': 0,
 u'upper_bound': u'excellent'}